Hate speech pada media sosial adalah suatu ungkapan yang sangat mengganggu.
Identifikasi hate speech secara otomatis akan membantu pihak terkait untuk melakukan suatu tindakan
secara efisien. Oleh sebab itu diperlukan bantuan dari anda untuk membangun model klasifikasi
multilabel Hate Speech pada data yang diberikan, dengan label **Hate Speech (HS), Abusive,
HS_Individual, HS_Group, HS_Religion, HS_Race,  HS_Physical, HS_Gender, HS_Other.**

### **Import lib**

In [2]:
!pip install transformers
!pip install --upgrade git+https://github.com/ariaghora/mpstemmer.git
!pip install Levenshtein

  Cloning https://github.com/ariaghora/mpstemmer.git to /tmp/pip-req-build-8qf9vxh8
  Running command git clone --filter=blob:none --quiet https://github.com/ariaghora/mpstemmer.git /tmp/pip-req-build-8qf9vxh8
  Resolved https://github.com/ariaghora/mpstemmer.git to commit 25a5fd923af163a7eac3a5ec976984156ca8fa8b
  Preparing metadata (setup.py) ... done
  Created wheel for mpstemmer: filename=mpstemmer-0.1.0-py3-none-any.whl size=99801 sha256=7d40d760b2136ff276f85c92d06d68bc45db44de1eaba9bf51583f4991c9c4dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-_ult_uzj/wheels/e0/b7/dd/9a80a3b7a05fe9e3252476ddf25da4faabddcec1b9e88fc32c
Successfully built mpstemmer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.4 MB/s eta 0:00:00


In [58]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

from textblob import TextBlob
import pandas as pd
import numpy as np
import os
import re
import string
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import BertTokenizer, AutoModelForMaskedLM, AutoModel

from mpstemmer import MPStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### **Soal**

##### Bangunlah model klasifikasi dengan menggunakan **Large Language Model (LLM)**, anda dapat menggunakan pretrained model yang akan anda jelaskan pada penjelasan dengan melakukan minimal 2 hyperparameter tuning. Dan paparkan hasil evaluasi metrik

In [4]:
# load dataset
df1 = pd.read_csv("data_1D.csv")
df1.head()

,Unnamed: 0,index,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,0,9320,USER wkwkwkw akhirnya antek Amerika ini pasang...,0,0,0,0,0,0,0,0,0
1,1,3964,Terlalu suuzon nih rezim URL,1,0,1,0,0,0,0,0,1
2,2,8551,"Mau tanya sama guntur romli, Bener gak neh :;",0,0,0,0,0,0,0,0,0
3,3,12366,Genosida Muslim Rohingya oleh Teroris Budha My...,1,0,0,1,1,0,0,0,0
4,4,6271,USER USER SubhanAllah sekolah di sekolah krist...,0,0,0,0,0,0,0,0,0


In [5]:
# data cleaning
# drop kolom tdk terpakai unnamed
df1_new = df1.drop(columns=['Unnamed: 0', 'index'])
df1_new.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,USER wkwkwkw akhirnya antek Amerika ini pasang...,0,0,0,0,0,0,0,0,0
1,Terlalu suuzon nih rezim URL,1,0,1,0,0,0,0,0,1
2,"Mau tanya sama guntur romli, Bener gak neh :;",0,0,0,0,0,0,0,0,0
3,Genosida Muslim Rohingya oleh Teroris Budha My...,1,0,0,1,1,0,0,0,0
4,USER USER SubhanAllah sekolah di sekolah krist...,0,0,0,0,0,0,0,0,0


### **Preprocessing Data**

In [6]:
# Preprocess data
# delete kata 'USER'
df1_new['clean_tweet'] = df1_new['Tweet'].apply(lambda x: re.sub("USER ", "", x))
df1_new.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,clean_tweet
0,USER wkwkwkw akhirnya antek Amerika ini pasang...,0,0,0,0,0,0,0,0,0,wkwkwkw akhirnya antek Amerika ini pasang bada...
1,Terlalu suuzon nih rezim URL,1,0,1,0,0,0,0,0,1,Terlalu suuzon nih rezim URL
2,"Mau tanya sama guntur romli, Bener gak neh :;",0,0,0,0,0,0,0,0,0,"Mau tanya sama guntur romli, Bener gak neh :;"
3,Genosida Muslim Rohingya oleh Teroris Budha My...,1,0,0,1,1,0,0,0,0,Genosida Muslim Rohingya oleh Teroris Budha My...
4,USER USER SubhanAllah sekolah di sekolah krist...,0,0,0,0,0,0,0,0,0,SubhanAllah sekolah di sekolah kristen namun i...


In [59]:
singkatan = pd.read_csv("singkatan.csv")
singkatan_dict = {}
for singk in singkatan.values:
    singkatan_dict[singk[0]] = singk[1]

def clean(text_text):
    # Mengubah menjadi huruf kecil
    clean_text = text_text.lower()
    # Menghilangkan Hashtag
    clean_text = re.sub(r'#\w+', "", clean_text)
    # Menghilangkan user mention
    clean_text = re.sub(r'@\w+', "", clean_text)
    # Menghilangkan Angka
    clean_text = re.sub(r'\d+', "", clean_text)

    tokenizer = RegexpTokenizer(r'\w+')
    # Menghilangkan tanda baca
    clean_text = " ".join(tokenizer.tokenize(clean_text))

  # Memperbaiki singkatan
    tokens = word_tokenize(clean_text)
    for i in range(len(tokens)):
        if tokens[i] in singkatan_dict.keys():
            tokens[i] = singkatan_dict[tokens[i]]

    filtered_tokens = [word for word in tokens if word not in stopwords.words('indonesian')]

    # Stemming Kata
    clean_text = ' '.join(filtered_tokens)

    return clean_text

df1_new["clean_tweet"] = df1_new["clean_tweet"].apply(lambda x: clean(x))
df1_new.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,clean_tweet
0,USER wkwkwkw akhirnya antek Amerika ini pasang...,0,0,0,0,0,0,0,0,0,wkwkwkw antek amerika pasang badan saham freep...
1,Terlalu suuzon nih rezim URL,1,0,1,0,0,0,0,0,1,suuzon nih rezim url
2,"Mau tanya sama guntur romli, Bener gak neh :;",0,0,0,0,0,0,0,0,0,guntur romli bener gak neh
3,Genosida Muslim Rohingya oleh Teroris Budha My...,1,0,0,1,1,0,0,0,0,genosida muslim rohingya teroris budha myanmar...
4,USER USER SubhanAllah sekolah di sekolah krist...,0,0,0,0,0,0,0,0,0,subhanallah sekolah sekolah kristen iman keper...


### **Tokenizer**

In [8]:
# pakai indobert
pretrained_model_name = "indobenchmark/indobert-large-p2"
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

### **Create Dataset**

In [9]:
class Dataset_loader(torch.utils.data.Dataset):
    def __init__(self, teks, labels, seq_len = 128):
        self.tokens = [tokenizer(sentence, padding="max_length",max_length=seq_len, truncation=True,return_tensors="pt") for sentence in tqdm(teks)]
        self.labels = labels
    def __len__(self):
        return len(self.tokens)
    def __getitem__(self, idx):
        return self.labels[idx], self.tokens[idx]

kelas Dataset_loader adalah menyediakan cara yang efisien untuk memuat dan memproses dataset teks untuk model pembelajaran mesin dengan PyTorch. Kelas ini melakukan tokenisasi pada teks, menambahkan padding, dan memotong kalimat sesuai dengan panjang maksimum yang ditentukan. Selain itu, kelas ini mengelola pengambilan data dengan menyediakan metode untuk mendapatkan panjang dataset (__len__) dan mengakses elemen dataset berdasarkan indeks (__getitem__). Hal ini memudahkan proses persiapan data untuk training model.

### **Splitting**

In [10]:
# Define
listlab = ['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion',
       'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']

x = df1_new['clean_tweet']
y = df1_new[listlab].values

In [11]:
# Split the data into training (70%), validation (15%), and test (15%) sets
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=42, test_size=0.50, shuffle=True)

In [12]:
# Print the lengths of each split
train_dataset = Dataset_loader(x_train, y_train)
val_dataset = Dataset_loader(x_val, y_val)
test_dataset = Dataset_loader(x_test, y_test)

  0%|          | 0/1152 [00:00<?, ?it/s]

  0%|          | 0/1153 [00:00<?, ?it/s]

  0%|          | 0/988 [00:00<?, ?it/s]

### **Build Model**

In [13]:
class indoBERT_classifier(nn.Module):
    def __init__(self, dropout, num_classes):
        super(indoBERT_classifier, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name, return_dict=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(1024, num_classes)
        self.activation = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        output_xlm = self.bert(input_ids=input_ids,
                                  attention_mask=attention_mask)
        dropout_output = self.dropout(output_xlm[1])
        final_output = self.linear(dropout_output)

        return final_output

Kelas **indoBERT_classifier** adalah sebuah model neural network yang dirancang untuk tugas klasifikasi teks, menggunakan arsitektur BERT yang telah dipra-latih. Dalam konstruktor (__init__), kelas ini menginisialisasi model BERT dari model pra-latih yang ditentukan (pretrained_model_name), dan menambahkan lapisan dropout untuk regularisasi dengan tingkat dropout yang ditentukan oleh parameter dropout. Selanjutnya, sebuah lapisan linear dengan ukuran input 1024 dan output yang sesuai dengan jumlah kelas (num_classes) ditambahkan untuk melakukan klasifikasi. Fungsi aktivasi ReLU juga diinisialisasi, meskipun tidak digunakan dalam forward pass.

Secara keseluruhan, kelas **indoBERT_classifier** memanfaatkan kekuatan model BERT yang telah dipra-latih untuk menghasilkan representasi teks yang kuat, dan menambahkan beberapa lapisan sederhana untuk melakukan tugas klasifikasi teks.

### **Train**

In [14]:
import torch
import numpy as np
from tqdm import tqdm

def accuracy_calculation(prediction, target):
    outputs = torch.sigmoid(prediction).cpu().detach().numpy().round()
    targets = target.cpu().detach().numpy()
    correct_predictions = np.sum(outputs == targets)
    num_samples = targets.size
    accuracy = float(correct_predictions) / num_samples
    return accuracy

def train(loader_train, loader_val, model, criterion, optimizer, device):
    train_loss = np.zeros(len(loader_train))
    train_accuracy = np.zeros(len(loader_train))
    val_loss = np.zeros(len(loader_val))
    val_accuracy = np.zeros(len(loader_val))

    model.train()
    for idx, (labels, data_batchs) in enumerate(tqdm(loader_train, total=len(loader_train))):
        input_ids = data_batchs["input_ids"][:, 0, :].to(device)
        attention_mask = data_batchs["attention_mask"][:, 0, :].to(device)
        labels = labels.to(device).float()

        output_model = model(input_ids, attention_mask)
        loss = criterion(output_model, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = accuracy_calculation(output_model, labels)

        train_loss[idx] = loss.item() / input_ids.size(0)
        train_accuracy[idx] = accuracy

    model.eval()
    with torch.no_grad():
        for idx, (labels, data_batchs) in enumerate(tqdm(loader_val, total=len(loader_val))):
            input_ids = data_batchs["input_ids"][:, 0, :].to(device)
            attention_mask = data_batchs["attention_mask"][:, 0, :].to(device)
            labels = labels.to(device).float()

            output_model = model(input_ids, attention_mask)
            loss = criterion(output_model, labels)
            accuracy = accuracy_calculation(output_model, labels)

            val_loss[idx] = loss.item() / input_ids.size(0)
            val_accuracy[idx] = accuracy

    log = {
        'train_loss': np.mean(train_loss),
        'train_accuracy': np.mean(train_accuracy),
        'val_loss': np.mean(val_loss),
        'val_accuracy': np.mean(val_accuracy)
    }

    return log


Kode ini bertujuan untuk melatih dan mengevaluasi model menggunakan dataset traiing (loader_train) dan validasi (loader_val). Fungsi accuracy_calculation menghitung akurasi prediksi model dengan membandingkan prediksi yang dibulatkan dengan nilai target yang sebenarnya. Fungsi train melatih model dengan melakukan forward pass, menghitung loss, dan memperbarui bobot model menggunakan backpropagation dan optimizer pada setiap batch data training.

In [15]:
# define si akurasi function
def accuracy_calculation(prediction, target):
    outputs = torch.sigmoid(prediction).cpu().detach().numpy().round()
    targets = target.cpu().detach().numpy()
    correct_predictions = np.sum(outputs == targets)
    num_samples = targets.size
    accuracy = float(correct_predictions) / num_samples
    return accuracy

 fungsi accuracy_calculation yang menghitung akurasi prediksi model pembelajaran mesin. Fungsi ini mengubah prediksi model menjadi probabilitas dengan fungsi sigmoid, kemudian membulatkannya untuk mendapatkan prediksi akhir. Prediksi ini dibandingkan dengan nilai target sebenarnya, dan jumlah prediksi yang benar dihitung. Akurasi kemudian dihitung sebagai rasio antara jumlah prediksi yang benar dengan total jumlah sampel, dan nilai akurasi ini dikembalikan oleh fungsi.

In [16]:
# define training function
def train(loader_train, loader_val, model, criterion, optimizer, device):
    train_loss = np.zeros(len(loader_train))
    train_accuracy = np.zeros(len(loader_train))
    val_loss = np.zeros(len(loader_val))
    val_accuracy = np.zeros(len(loader_val))

    model.train()
    for idx, (labels, data_batchs) in enumerate(tqdm(loader_train, total=len(loader_train))):
        input_ids = data_batchs["input_ids"][:, 0, :].to(device)
        attention_mask = data_batchs["attention_mask"][:, 0, :].to(device)
        labels = labels.to(device).float()

        output_model = model(input_ids, attention_mask)
        loss = criterion(output_model, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = accuracy_calculation(output_model, labels)

        train_loss[idx] = loss.item() / input_ids.size(0)
        train_accuracy[idx] = accuracy

    model.eval()
    with torch.no_grad():
        for idx, (labels, data_batchs) in enumerate(tqdm(loader_val, total=len(loader_val))):
            input_ids = data_batchs["input_ids"][:, 0, :].to(device)
            attention_mask = data_batchs["attention_mask"][:, 0, :].to(device)
            labels = labels.to(device).float()

            output_model = model(input_ids, attention_mask)
            loss = criterion(output_model, labels)
            accuracy = accuracy_calculation(output_model, labels)

            val_loss[idx] = loss.item() / input_ids.size(0)
            val_accuracy[idx] = accuracy

    log = {
        'train_loss': np.mean(train_loss),
        'train_accuracy': np.mean(train_accuracy),
        'val_loss': np.mean(val_loss),
        'val_accuracy': np.mean(val_accuracy)
    }

    return log


Fungsi train melakukan proses training dengan mengiterasi batch data pelatihan, melakukan forward pass untuk mendapatkan prediksi model, menghitung loss menggunakan fungsi loss yang diberikan, dan memperbarui bobot model melalui backpropagation. Selama training, akurasi dan loss untuk setiap batch dicatat. Setelah itu, fungsi ini beralih ke mode evaluasi dan menghitung akurasi serta loss pada data validasi tanpa melakukan backpropagation.

In [30]:
# define main function
def main(train_data, val_data, EPOCHS, learning_r, device, BATCH_SIZE, dropout=0.3):
    criterion = torch.nn.BCEWithLogitsLoss()
    model = indoBERT_classifier(dropout=dropout, num_classes=len(listlab))
    model = model.to(device)

    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_r, weight_decay=0.0001)

    history_log = {
        'epoch': [],
        'lr': [],
        'loss': [],
        'accuracy': [],
        'val_loss': [],
        'val_accuracy': []
    }

    loader_train = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
    loader_val = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, pin_memory=False)

    for epoch in range(EPOCHS):
        print('Epoch [%d/%d]' % (epoch + 1, EPOCHS))

        torch.cuda.empty_cache()
        log = train(loader_train=loader_train, loader_val=loader_val, model=model, criterion=criterion, optimizer=optimizer, device=device)

        print(f'loss {log["train_loss"]:.4f} - accuracy {log["train_accuracy"]:.4f} - val_loss {log["val_loss"]:.4f} - val_accuracy {log["val_accuracy"]:.4f}')

        history_log['epoch'].append(epoch + 1)
        history_log['lr'].append(learning_r)
        history_log['loss'].append(log['train_loss'])
        history_log['accuracy'].append(log['train_accuracy'])
        history_log['val_loss'].append(log['val_loss'])
        history_log['val_accuracy'].append(log['val_accuracy'])

        # Define the path to save the model in the local Colab folder
        save_model_path = f'model_weights_{EPOCHS}_{dropout}_{learning_r}.pth'

        # Save the model state dictionary
        torch.save(model.state_dict(), save_model_path)

        # Print confirmation message
        print(f"Model saved to {save_model_path}")
        return history_log


### **Hyperparameter Tuning (manually)**

In [48]:
history_train = main(train_dataset,
                     val_dataset,
                     EPOCHS = 5,
                     learning_r = 3e-5,
                     BATCH_SIZE = 32,
                     device = 'cuda',
                     dropout=0.3
                     )

Epoch [1/5]


100%|██████████| 37/37 [00:24<00:00,  1.52it/s]


loss 0.0141 - accuracy 0.7814 - val_loss 0.0159 - val_accuracy 0.8629
Model saved to model_weights_5_0.3_3e-05.pth


In [38]:
history_train = main(train_dataset,
                     val_dataset,
                     EPOCHS = 10,
                     learning_r = 3e-5,
                     BATCH_SIZE = 32,
                     device = 'cuda',
                     dropout=0.3
                     )

Epoch [1/10]


100%|██████████| 37/37 [00:24<00:00,  1.51it/s]


loss 0.0138 - accuracy 0.7942 - val_loss 0.0145 - val_accuracy 0.8645
Model saved to model_weights_10_0.3_3e-05.pth


In [49]:
history_train = main(train_dataset,
                     val_dataset,
                     EPOCHS = 5,
                     learning_r = 5e-5,
                     BATCH_SIZE = 32,
                     device = 'cuda',
                     dropout=0.3
                     )

Epoch [1/5]


100%|██████████| 37/37 [00:24<00:00,  1.53it/s]


loss 0.0123 - accuracy 0.8258 - val_loss 0.0124 - val_accuracy 0.8880
Model saved to model_weights_5_0.3_5e-05.pth


In [50]:
history_train = main(train_dataset,
                     val_dataset,
                     EPOCHS = 10,
                     learning_r = 5e-5,
                     BATCH_SIZE = 32,
                     device = 'cuda',
                     dropout=0.3
                     )

Epoch [1/10]


100%|██████████| 37/37 [00:24<00:00,  1.53it/s]


loss 0.0126 - accuracy 0.8156 - val_loss 0.0121 - val_accuracy 0.8910
Model saved to model_weights_10_0.3_5e-05.pth


Berdasarkan hasil dari empat hyperparameter yg diuji. <BR>
- **Learning Rate:**
Pada semua epoch, learning rate yang lebih tinggi (5e-5) memberikan akurasi validasi yang lebih baik dibandingkan dengan learning rate yang lebih rendah (3e-5). Ini menunjukkan bahwa model dapat belajar lebih efektif dengan learning rate yang lebih tinggi dalam range ini.

- **Epochs:** Jumlah epochs dari 5 ke 10 cenderung meningkatkan akurasi validasi dan menurunkan loss validasi. Artinya, model mendapat manfaat dari pelatihan lebih lama, meskipun peningkatan performa dari 5 epochs ke 10 epochs tidak terlalu signifikan pada learning rate yang lebih tinggi.

- **Overfitting:** No indication, karena baik training loss dan validation loss tetap menurun dengan penambahan epochs. Tetapi, model dengan learning rate yang lebih tinggi cenderung memiliki training accuracy yang lebih tinggi.

- **Optimal Hyperparameter:**
Berdasarkan hasil ini, kombinasi optimal tampaknya adalah Epochs = 10 dan Learning Rate = 5e-5, yang memberikan akurasi validasi tertinggi (89.10%) dan loss validasi terendah (0.0121).

### **Testing**

In [51]:
def testing(loader_test,
          model,
          criterion,
          device):
    loss_all_test = np.zeros(len(loader_test))
    accuracy_all_test = np.zeros(len(loader_test))
    predictions = []
    target_values = []

    model.to(device)
    model.eval()

    idx = 0
    with torch.no_grad():
        for labels, batch_data in tqdm(loader_test, total=len(loader_test)):
            input_ids = batch_data["input_ids"][:,0,:]
            attention_mask = batch_data["attention_mask"][:,0,:]

            labels = labels.to(device).float()
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            # Forward pass
            output_model = model(input_ids, attention_mask)

            loss = criterion(output_model, labels)

            accuracy = accuracy_calculation(output_model,labels)

            pred_prob = torch.sigmoid(output_model).cpu().detach()
            predictions.extend(pred_prob.round())
            target_values.extend(labels.cpu().detach())

            loss_all_test[idx] = loss.item()/input_ids.size(0)
            accuracy_all_test[idx] = accuracy
            idx += 1

    log = {
        'loss':np.mean(loss_all_test),
        'accuracy':np.mean(accuracy_all_test)
    }

    predictions = torch.stack(predictions)
    target_values = torch.stack(target_values)

    print(f"Test loss {np.mean(loss_all_test)} - accuracy {np.mean(accuracy_all_test)}")

    return predictions, target_values, np.mean(accuracy_all_test)

Function testing ini dirancang untuk mengevaluasi kinerja model menggunakan dataset test. Function ini menerima empat parameter: loader_test (DataLoader untuk data test), model (model yang akan dievaluasi), criterion (Function loss), dan device (CPU atau GPU). Function ini menginisialisasi array untuk menyimpan loss dan akurasi setiap batch, serta daftar untuk menyimpan prediksi dan nilai target. Model diatur ke mode evaluasi dan dipindahkan ke perangkat yang ditentukan. Function ini kemudian melakukan iterasi melalui batch data uji, memindahkan data input dan label ke perangkat, melakukan forward pass untuk mendapatkan prediksi model, serta menghitung loss dan akurasi untuk setiap batch. Prediksi dan label dikumpulkan, dan rata-rata loss serta akurasi dihitung dan dicetak. Function ini mengembalikan prediksi, nilai target, dan rata-rata akurasi, yang memungkinkan evaluasi kinerja model secara efisien pada data uji.

In [52]:
# import lib dan inisialisasi parameter
import warnings
warnings.filterwarnings('ignore')

epoch_lst = [5, 10]
dropout_on = 0.3
learning_r_lst = [3e-5, 5e-5]
device = "cuda"

loss_fn = torch.nn.BCEWithLogitsLoss()

dict_log = {
    'dropout': [],
    'epoch': [],
    'learning_rate': [],
    'acc_train': [],
    'f1_train': [],
    'precision_train': [],
    'recall_train': [],
    'acc_val': [],
    'f1_val': [],
    'precision_val': [],
    'recall_val': [],
    'acc_test': [],
    'f1_test': [],
    'precision_test': [],
    'recall_test': []
}

Code ini berfungsi untuk mempersiapkan lingkungan dan parameter yang dibutuhkan untuk melatih dan mengevaluasi model. Nilai epoch (epoch_lst) dan learning rate (learning_r_lst) diinisialisasi dengan nilai-nilai tertentu, yaitu [5, 10] untuk epoch dan [3e-5, 5e-5] untuk learning rate. Parameter dropout ditetapkan sebesar 0.3 (dropout_on). Kode ini juga mengatur penggunaan GPU dengan menetapkan device ke "cuda". Fungsi kehilangan (loss function) yang digunakan adalah BCEWithLogitsLoss, cocok untuk tugas klasifikasi biner dengan output logits. Terakhir, kode ini membuat dictionary (dict_log) untuk mencatat hasil evaluasi model,

In [53]:
# data loader inisialisasi
loader_test = DataLoader(test_dataset, batch_size=32)
loader_train = DataLoader(train_dataset, batch_size=32)
loader_val = DataLoader(val_dataset, batch_size=32)

In [57]:
# looping
for epoch_on in epoch_lst:
    for learning_r in learning_r_lst:
        save_path = f"model_weights_{epoch_on}_{dropout_on}_{learning_r}.pth"
        print(save_path)

        model_test = indoBERT_classifier(dropout=dropout_on, num_classes=len(listlab))
        checkpoint = torch.load(save_path)
        model_test.load_state_dict(checkpoint)
        model_test = model_test.to(device)

        # Training Data
        print("Training")
        predictions_train, target_values_train, accuracy_train = testing(loader_test=loader_train,
                                                                         model=model_test,
                                                                         criterion=loss_fn,
                                                                         device=device)
        predictions_train = predictions_train.numpy()
        target_values_train = target_values_train.numpy()

        f1_score_train = f1_score(target_values_train, predictions_train, average='micro')
        precision_score_train = precision_score(target_values_train, predictions_train, average='micro')
        recall_score_train = recall_score(target_values_train, predictions_train, average='micro')

        # Validation Data
        print("Validasi")
        predictions_val, target_values_val, accuracy_val = testing(loader_test=loader_val,
                                                                   model=model_test,
                                                                   criterion=loss_fn,
                                                                   device=device)
        predictions_val = predictions_val.numpy()
        target_values_val = target_values_val.numpy()

        f1_score_val = f1_score(target_values_val, predictions_val, average='micro')
        precision_score_val = precision_score(target_values_val, predictions_val, average='micro')
        recall_score_val = recall_score(target_values_val, predictions_val, average='micro')

        # Testing Data
        print("Testing")
        predictions_test, target_values_test, accuracy_test = testing(loader_test=loader_test,
                                                                      model=model_test,
                                                                      criterion=loss_fn,
                                                                      device=device)
        predictions_test = predictions_test.numpy()
        target_values_test = target_values_test.numpy()

        f1_score_test = f1_score(target_values_test, predictions_test, average='micro')
        precision_score_test = precision_score(target_values_test, predictions_test, average='micro')
        recall_score_test = recall_score(target_values_test, predictions_test, average='micro')

        # Logging Results
        print("\n")
        print("---" * 160)
        print("\n")

        dict_log['epoch'].append(epoch_on)
        dict_log['dropout'].append(dropout_on)
        dict_log['learning_rate'].append(learning_r)
        dict_log['acc_train'].append(accuracy_train)
        dict_log['precision_train'].append(precision_score_train)
        dict_log['recall_train'].append(recall_score_train)
        dict_log['f1_train'].append(f1_score_train)
        dict_log['acc_val'].append(accuracy_val)
        dict_log['precision_val'].append(precision_score_val)
        dict_log['recall_val'].append(recall_score_val)
        dict_log['f1_val'].append(f1_score_val)
        dict_log['acc_test'].append(accuracy_test)
        dict_log['precision_test'].append(precision_score_test)
        dict_log['recall_test'].append(recall_score_test)
        dict_log['f1_test'].append(f1_score_test)

dict_log_test = pd.DataFrame(dict_log)
dict_log_test


model_weights_5_0.3_3e-05.pth
Training


100%|██████████| 36/36 [00:21<00:00,  1.64it/s]


Test loss 0.010754653200921085 - accuracy 0.861400462962963
Validasi


100%|██████████| 37/37 [00:22<00:00,  1.65it/s]


Test loss 0.01590574552883973 - accuracy 0.8628941441441441
Testing


...

100%|██████████| 37/37 [00:24<00:00,  1.53it/s]


Test loss 0.012088192639419355 - accuracy 0.8910472972972973
Testing


100%|██████████| 31/31 [00:20<00:00,  1.49it/s]

Test loss 0.009610118666311838 - accuracy 0.8750480030721967


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------




,dropout,epoch,learning_rate,acc_train,f1_train,precision_train,recall_train,acc_val,f1_val,precision_val,recall_val,acc_test,f1_test,precision_test,recall_test
0,0.3,5,0.00003,0.861400,0.465203,0.835561,0.322331,0.862894,0.458743,0.782946,0.324411,0.850886,0.435559,0.791345,0.300469
1,0.3,5,0.00005,0.898630,0.715099,0.753714,0.680248,0.887950,0.664229,0.699645,0.632227,0.874024,0.646725,0.697690,0.602700
2,0.3,10,0.00003,0.863812,0.534301,0.741080,0.417741,0.864489,0.512821,0.693431,0.406852,0.854023,0.513665,0.709411,0.402582
3,0.3,10,0.00005,0.893133,0.675264,0.782077,0.594121,0.891047,0.639105,0.762046,0.550321,0.875048,0.617343,0.746256,0.526408
4,0.3,5,0.00003,0.861400,0.465203,0.835561,0.322331,0.862894,0.458743,0.782946,0.324411,0.850886,0.435559,0.791345,0.300469
5,0.3,5,0.00005,0.898630,0.715099,0.753714,0.680248,0.887950,0.664229,0.699645,0.632227,0.874024,0.646725,0.697690,0.602700
6,0.3,10,0.00003,0.863812,0.534301,0.741080,0.417741,0.864489,0.512821,0.693431,0.406852,0.854023,0.513665,0.709411,0.402582
7,0.3,10,0.00005,0.893133,0.675264,0.782077,0.594121,0.891047,0.639105,0.762046,0.550321,0.875048,0.617343,0.746256,0.526408


Code ini berfungsi untuk mengevaluasi model machine learning dengan berbagai kombinasi parameter epoch dan learning rate. Dalam setiap iterasi, model ditraining dan ditesting menggunakan parameter yang berbeda. Pertama, kode initialize model indoBERT_classifier dengan dropout 0.3 dan memuat model dari file yang disimpan di colab folder. Kemudian, model dievaluasi menggunakan data training, validasi, dan testing, menghasilkan metrics seperti akurasi, f1 score, precision, dan recall untuk masing-masing dataset. Hasil evaluasi ini dicatat dalam dictionary dict_log, yang akhirnya dikonversi menjadi DataFrame dict_log_test untuk next analisis. Proses ini membantu dalam performance model, sehingga dapat memilih parameter yang optimal untuk hasil terbaik.

### **Conlusion**

- Learning Rate = 5e-5, Epochs = 5
Memiliki training accuracy tertinggi (89.86%) dan validation accuracy kedua tertinggi (88.79%). Testing accuracy kedua tertinggi (87.40%). F1 score juga tinggi di semua metrik (training, validation, testing).

- Learning Rate = 5e-5, Epochs = 10 Memiliki validation accuracy tertinggi (89.10%) dan testing accuracy tertinggi (87.50%). F1 score yang tinggi di semua metrik menunjukkan model ini berkinerja baik secara keseluruhan.

- Learning Rate = 3e-5, Epochs = 10
Memiliki accuracy terendah di antara semua kombinasi untuk semua metrik (training, validation, dan testing). F1 score juga relatif rendah di semua metrik.


**Kombinasi Optimal:**

Kombinasi terbaik dari hyperparameter adalah Learning Rate = 5e-5 dan Epochs = 10, yang memberikan validation accuracy tertinggi (89.10%) dan testing accuracy tertinggi (87.50%).
Selain itu, F1 score yang tinggi menunjukkan keseimbangan yang baik antara precision dan recall.

**Learning Rate:**
Learning rate yang lebih tinggi (5e-5) lebih baik dibandingkan learning rate yang lebih rendah (3e-5) dalam skenario ini.

**Epochs:**
Menambah jumlah epochs dari 5 ke 10 meningkatkan kinerja model, terutama pada learning rate yang lebih tinggi.